In [158]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest ,chi2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

In [159]:
df = pd.read_csv('updated_mobile_info.csv')

In [160]:
df.head()

,Unnamed: 0,name,ratings,number of rating,number of reviews,ram,rom,expandable,display,camera,battery,price
0,0,Cellecor E3,3.7,56.0,2.0,32.0,32.0,32.0,1.80,0.8,1000.0,1061.0
1,1,"REDMI 10 (Caribbean Green, 64 GB)",4.3,222897.0,13810.0,4.0,64.0,1.0,6.70,50.0,6000.0,9499.0
2,2,"REDMI 10 (Midnight Black, 64 GB)",4.3,222897.0,13810.0,4.0,64.0,1.0,6.70,50.0,6000.0,9499.0
3,3,"MOTOROLA G32 (Satin Silver, 128 GB)",4.1,9683.0,799.0,8.0,128.0,NaN,6.55,50.0,5000.0,11999.0
4,4,"realme C33 2023 (Night Sea, 64 GB)",4.5,6542.0,184.0,4.0,64.0,1.0,6.50,50.0,5000.0,9999.0


In [161]:
df = df.replace(0,5)

In [162]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               586 non-null    object 
 1   ratings            580 non-null    float64
 2   number of rating   586 non-null    float64
 3   number of reviews  586 non-null    float64
 4   ram                582 non-null    float64
 5   rom                559 non-null    float64
 6   expandable         331 non-null    float64
 7   display            585 non-null    float64
 8   camera             586 non-null    float64
 9   battery            565 non-null    float64
 10  price              586 non-null    float64
dtypes: float64(10), object(1)
memory usage: 50.5+ KB


In [164]:
df.drop('name',axis=1,inplace=True)

In [165]:
df.head()

,ratings,number of rating,number of reviews,ram,rom,expandable,display,camera,battery,price
0,3.7,56.0,2.0,32.0,32.0,32.0,1.80,0.8,1000.0,1061.0
1,4.3,222897.0,13810.0,4.0,64.0,1.0,6.70,50.0,6000.0,9499.0
2,4.3,222897.0,13810.0,4.0,64.0,1.0,6.70,50.0,6000.0,9499.0
3,4.1,9683.0,799.0,8.0,128.0,NaN,6.55,50.0,5000.0,11999.0
4,4.5,6542.0,184.0,4.0,64.0,1.0,6.50,50.0,5000.0,9999.0


In [166]:
df.isnull().sum()

ratings                6
number of rating       0
number of reviews      0
ram                    4
rom                   27
expandable           255
display                1
camera                 0
battery               21
price                  0
dtype: int64

In [167]:
x_train.head()

,ratings,number of rating,number of reviews,ram,rom,expandable,display,camera,battery
527,3.8,2199.0,231.0,64.0,64.0,NaN,3.50,1.3,6000.0
173,4.0,955.0,85.0,4.0,4.0,32.0,1.80,5.0,1000.0
453,4.5,144.0,19.0,8.0,128.0,NaN,6.78,64.0,5000.0
542,3.3,6980.0,756.0,1.0,8.0,32.0,4.00,5.0,1600.0
55,4.6,32424.0,1255.0,128.0,NaN,NaN,6.10,12.0,NaN


In [168]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('price',axis=1),
                                                df['price'],
                                                test_size=0.15,
                                                random_state=0)

In [200]:
# Function transformer
trf2 = ColumnTransformer(transformers=[
    ('n1',FunctionTransformer(func=np.log),[0,1,3,4,5,6,7,8]),
],remainder='passthrough')

In [201]:
trf1 =ColumnTransformer(transformers=[
    ('n3',SimpleImputer(),[0,3,4,5,6,8])
],remainder='passthrough')

In [202]:
trf4 =LinearRegression()

In [203]:
#trf3= ColumnTransformer(transformers=[
 #  ('n4',StandardScaler(),[9])
#],remainder='passthrough')

In [204]:
pipe = Pipeline([
    ('trf1',trf1),
    #('trf2',trf3),
    ('trf3',trf2),
    ('trf4',trf4)
])

In [205]:

from sklearn.compose import TransformedTargetRegressor
wrapped_model = TransformedTargetRegressor(regressor=pipe, transformer=StandardScaler())

In [206]:
wrapped_model.fit(x_train, y_train)
yhat = wrapped_model.predict(x_test)

In [207]:
r2_score(yhat,y_test)

0.5046414436208102

In [194]:
y_pred = pipe.predict(x_test)

In [189]:
r2_score(y_pred,y_test)

0.5046414436208202

In [177]:
df['price']

0       1061.0
1       9499.0
2       9499.0
3      11999.0
4       9999.0
        ...   
581      999.0
582    10999.0
583     9990.0
584     1279.0
585     1304.0
Name: price, Length: 586, dtype: float64